In [19]:
library(tsfeatures)
library(imputeTS)
library(forecast)
library(dplyr)

In [20]:
data=read.csv('Imports.csv')

In [21]:
head(data)

,月份,日本,经合组织成员国,OECD成员国_欧洲,OECD和六个非成员经济体,主要亚洲五国.OECD.,瑞士,韩国,土耳其,比利时,⋯,中华人民共和国,南非,哥伦比亚,墨西哥,新西兰,以色列,哥斯达黎加,欧元区,俄罗斯,爱尔兰
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1995-01,100.86,100.98,100.79,100.34,100.74,99.21,NA,NA,100.50,⋯,101.44,101.91,NA,NA,101.35,NA,NA,100.62,NA,102.04
2,1995-02,100.60,100.85,100.73,100.26,100.78,98.84,NA,NA,100.42,⋯,101.95,101.79,NA,NA,101.32,NA,NA,100.56,NA,102.02
3,1995-03,100.34,100.71,100.69,100.15,100.80,98.80,NA,NA,100.46,⋯,102.36,101.73,NA,NA,101.38,NA,NA,100.57,NA,101.89
4,1995-04,100.14,100.66,100.75,100.11,100.79,98.97,NA,NA,100.47,⋯,102.63,101.73,NA,NA,101.55,NA,NA,100.67,NA,101.70
5,1995-05,100.05,100.69,100.85,100.12,100.77,99.18,NA,NA,100.46,⋯,102.72,101.78,NA,NA,101.77,NA,NA,100.78,NA,101.67
6,1995-06,100.04,100.78,100.97,100.18,100.73,99.33,NA,NA,100.44,⋯,102.63,101.83,NA,NA,101.94,NA,NA,100.80,NA,101.76


In [22]:
tail(data)

,月份,日本,经合组织成员国,OECD成员国_欧洲,OECD和六个非成员经济体,主要亚洲五国.OECD.,瑞士,韩国,土耳其,比利时,⋯,中华人民共和国,南非,哥伦比亚,墨西哥,新西兰,以色列,哥斯达黎加,欧元区,俄罗斯,爱尔兰
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
341,2023-05,98.40,98.30,98.63,97.17,95.11,96.91,99.64,99.75,99.31,⋯,93.33,96.46,95.95,103.91,96.67,97.84,101.81,98.16,99.61,NA
342,2023-06,98.53,98.49,98.73,97.21,94.89,96.99,100.02,98.74,99.40,⋯,92.96,96.54,96.52,104.36,96.79,97.71,102.00,98.39,NA,NA
343,2023-07,98.57,98.59,98.61,97.26,94.81,97.04,100.26,96.99,99.56,⋯,92.84,96.85,96.68,104.87,96.78,97.74,102.38,NA,NA,NA
344,2023-08,98.48,98.50,98.36,97.22,94.82,96.93,100.24,95.26,99.63,⋯,92.88,97.28,96.64,105.27,96.70,97.62,102.82,NA,NA,NA
345,2023-09,98.35,98.33,98.13,97.14,94.86,96.56,99.99,94.77,99.60,⋯,93.00,97.76,96.59,105.55,96.58,NA,NA,NA,NA,NA
346,2023-10,98.27,98.09,97.90,96.90,NA,96.02,99.69,94.95,99.55,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [23]:
data=data[1:285,c(2:dim(data)[2])]

In [24]:
dim(data)

[1] 345  46

In [25]:
which_baddata=function(data){
baddata=c()
for(i in 1:dim(data)[2])
{loc=min(which(is.na(data[,i])==FALSE))-1
if((dim(data)[1]-loc)<40)
    {
    baddata=append(baddata,i)
}
if(sum(is.na(data[loc:dim(data)[1],i])==TRUE)>10)
 {
     baddata=append(baddata,i)
 }
 if(sum(is.na(data[(dim(data)[1]-11):dim(data)[1],i])==TRUE)>3)
 {
     baddata=append(baddata,i)
 }
}
baddata=unique(baddata)
return(baddata)}

In [26]:
baddata=which_baddata(data)

In [27]:
baddata

[1] 45 46

In [28]:
data=data[,-1*baddata]

In [29]:
data

,日本,经合组织成员国,OECD成员国_欧洲,OECD和六个非成员经济体,主要亚洲五国.OECD.,瑞士,韩国,土耳其,比利时,丹麦,⋯,西方七国,印尼,中华人民共和国,南非,哥伦比亚,墨西哥,新西兰,以色列,哥斯达黎加,欧元区
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,100.86,100.98,100.79,100.34,100.74,99.21,NA,NA,100.50,101.19,⋯,100.79,NA,101.44,101.91,NA,NA,101.35,NA,NA,100.62
2,100.60,100.85,100.73,100.26,100.78,98.84,NA,NA,100.42,101.24,⋯,100.67,NA,101.95,101.79,NA,NA,101.32,NA,NA,100.56
3,100.34,100.71,100.69,100.15,100.80,98.80,NA,NA,100.46,101.22,⋯,100.51,NA,102.36,101.73,NA,NA,101.38,NA,NA,100.57
4,100.14,100.66,100.75,100.11,100.79,98.97,NA,NA,100.47,101.16,⋯,100.46,NA,102.63,101.73,NA,NA,101.55,NA,NA,100.67
5,100.05,100.69,100.85,100.12,100.77,99.18,NA,NA,100.46,101.09,⋯,100.49,NA,102.72,101.78,NA,NA,101.77,NA,NA,100.78
6,100.04,100.78,100.97,100.18,100.73,99.33,NA,NA,100.44,101.01,⋯,100.60,NA,102.63,101.83,NA,NA,101.94,NA,NA,100.80
7,100.10,100.86,101.00,100.21,100.68,99.41,NA,NA,100.27,101.03,⋯,100.69,NA,102.40,101.89,NA,NA,102.02,NA,NA,100.72
8,100.24,100.83,100.94,100.17,100.65,99.46,NA,NA,99.91,101.13,⋯,100.66,NA,102.14,101.95,NA,NA,102.01,NA,NA,100.56
9,100.41,100.70,100.80,100.03,100.66,99.47,NA,NA,99.50,101.18,⋯,100.50,NA,101.93,102.03,NA,NA,101.93,NA,NA,100.34


In [30]:
freq=12
ll=dim(data)[2]
hh=12
start=c(2000,1)
m=5
n=4

In [31]:
process_data <- function(data, start, freq, hh, ll) {
  data_ts <- list()
  data_tstrain <- list()
  data_features <- NULL
  
  for (i in 1:ll) {
    # 寻找第一个非NA值的位置
    loc <- min(which(is.na(data[, i]) == FALSE)) - 1
    zc <- loc %/% freq
    yu <- loc %% freq
    
    # 提取子序列
    x <- data[(loc + 1):dim(data)[1], i]
    
    # 检测并处理NA值
    r <- which(is.na(x) == TRUE)
    if (length(r) != 0) {
      x <- na_kalman(x) # 假设na_kalman是一个处理NA值的函数
    }
    
    # 创建时间序列对象
    da <- ts(x[1:(length(x) - hh)], start = c(start[1] + zc, start[2] + yu), frequency = freq)
    da1 <- ts(x, start = c(start[1] + zc, start[2] + yu), frequency = freq)
    
    # 保存时间序列对象
    data_ts[[i]] <- da1
    data_tstrain[[i]] <- da
    
    # 提取时间序列特征
    f <- tsfeatures(da)
    if (is.null(data_features)) {
      data_features <- f
    } else {
      data_features <- rbind(data_features, f)
    }
  }
  
  # 函数返回一个列表，包含处理后的时间序列和特征
  list(time_series = data_ts, time_series_train = data_tstrain, features = data_features)
}


In [32]:
processed_data <- process_data(data, start = c(1995, 1), freq, hh, ll)

In [33]:
data_ts=processed_data$time_series
data_tstrain=processed_data$time_series_train
data_features=processed_data$features

In [34]:
head(data_features)

frequency,nperiods,seasonal_period,trend,spike,linearity,curvature,e_acf1,e_acf10,seasonal_strength,peak,trough,entropy,x_acf1,x_acf10,diff1_acf1,diff1_acf10,diff2_acf1,diff2_acf10,seas_acf1
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
12,1,12,0.9414541,1.469496e-07,-4.148065,-3.765981,0.8454210,1.159314,0.12604369,12,5,0.5977804,0.9774598,5.517595,0.8238031,0.9675078,0.5318010,0.9109732,0.3526531
12,1,12,0.9630601,8.844036e-08,-8.774571,2.937463,0.8582569,1.183075,0.09641165,1,5,0.5427694,0.9757847,6.052019,0.8190963,0.9426224,0.4867796,0.6482644,0.5042263
12,1,12,0.9656192,5.579385e-08,-8.151818,1.956549,0.8807846,1.410732,0.10045470,9,4,0.5458390,0.9712250,5.603514,0.8506849,1.2190287,0.4892918,0.5528145,0.4555680
12,1,12,0.9531315,1.222234e-07,-3.941638,1.944559,0.8604031,1.131996,0.24430725,1,5,0.5882649,0.9620671,4.573657,0.8462332,1.0444014,0.5709416,0.8613847,0.3172945
12,1,12,0.9492339,1.599164e-07,3.179791,5.670570,0.8527939,1.177701,0.39023178,1,6,0.6173344,0.9634733,4.700404,0.8277382,1.0134493,0.5072502,0.6229488,0.4188087
12,1,12,0.9320480,1.708380e-07,-2.018620,-6.632152,0.8763513,1.222079,0.25096789,11,5,0.6383962,0.9543605,3.877693,0.8660764,1.1364421,0.6677847,0.9972327,0.1575627


In [35]:
long=matrix(0,ll,1)
for( i in 1:ll)
    {
    long[i,1]=length(data_tstrain[[i]])
}

In [36]:
data_features=cbind(data_features,long)

In [37]:
head(data_features)

,frequency,nperiods,seasonal_period,trend,spike,linearity,curvature,e_acf1,e_acf10,seasonal_strength,⋯,trough,entropy,x_acf1,x_acf10,diff1_acf1,diff1_acf10,diff2_acf1,diff2_acf10,seas_acf1,long
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,12,1,12,0.9414541,1.469496e-07,-4.148065,-3.765981,0.8454210,1.159314,0.12604369,⋯,5,0.5977804,0.9774598,5.517595,0.8238031,0.9675078,0.5318010,0.9109732,0.3526531,333
2,12,1,12,0.9630601,8.844036e-08,-8.774571,2.937463,0.8582569,1.183075,0.09641165,⋯,5,0.5427694,0.9757847,6.052019,0.8190963,0.9426224,0.4867796,0.6482644,0.5042263,333
3,12,1,12,0.9656192,5.579385e-08,-8.151818,1.956549,0.8807846,1.410732,0.10045470,⋯,4,0.5458390,0.9712250,5.603514,0.8506849,1.2190287,0.4892918,0.5528145,0.4555680,333
4,12,1,12,0.9531315,1.222234e-07,-3.941638,1.944559,0.8604031,1.131996,0.24430725,⋯,5,0.5882649,0.9620671,4.573657,0.8462332,1.0444014,0.5709416,0.8613847,0.3172945,333
5,12,1,12,0.9492339,1.599164e-07,3.179791,5.670570,0.8527939,1.177701,0.39023178,⋯,6,0.6173344,0.9634733,4.700404,0.8277382,1.0134493,0.5072502,0.6229488,0.4188087,333
6,12,1,12,0.9320480,1.708380e-07,-2.018620,-6.632152,0.8763513,1.222079,0.25096789,⋯,5,0.6383962,0.9543605,3.877693,0.8660764,1.1364421,0.6677847,0.9972327,0.1575627,333


In [38]:
f1=function(x){
    if( x%%1>=0.5)
        {
        return(as.integer(x)+1)
    }
    else{return(as.integer(x))}
}

In [39]:
forecast_and_evaluate <- function(data_ts, data_tstrain, m,n,hh, ll) {
  predets <- array(0, dim = c(ll, m, n, 6))
  predhets <- array(0, dim = c(ll, m, n, hh))
  predthetaf <- array(0, dim = c(ll, m, n, 6))
  predhthetaf <- array(0, dim = c(ll, m, n, hh))
  yreal <- list()
  
  for (k in 1:ll) {
    y <- data_tstrain[[k]]
    yall <- data_ts[[k]]
    y_pred <- ts(yall[(length(yall) - hh + 1):length(yall)], end = end(yall), frequency = frequency(yall))
    y_l=length(y)
    loc = 1:length(y)
    loc_m = as.integer(loc*m/length(y))
    filt_d = data.frame(y,loc_m)
    filt_0=filter(filt_d ,loc_m==0)
    m_l=count(filt_0)
    n_l=m_l/n
    m_l=as.integer(m_l)
    n_l=as.numeric(n_l)
    
    yreal[[k]] <- y_pred
    
    for (i in 0:(m - 1)) {
      for (j in 0:(n-1)) {
        Y <- ts(y[round((i*m_l+j*n_l)+1):y_l], end = end(y), frequency = frequency(yall))
        M <- ets(Y)
        pd <- forecast(M, h = hh)
        predhets[k, (i + 1), (j + 1), ] <- pd$mean
        predets[k, (i + 1), (j + 1), ] <- accuracy(pd, y_pred)[2, 1:6]
        
        M <- thetaf(Y, h = hh)
        pd <- forecast(M, h = hh)
        predhthetaf[k, (i + 1), (j + 1), ] <- pd$mean
        predthetaf[k, (i + 1), (j + 1), ] <- accuracy(pd, y_pred)[2, 1:6]
      }
    }
  }
  
  list(
    pred_ets = list(predhets = predhets, predets = predets),
    pred_thetaf = list(predhthetaf = predhthetaf, predthetaf = predthetaf),
    y_real = yreal
  )
}

In [40]:
forecast_list=forecast_and_evaluate(data_ts, data_tstrain, m,n,hh, ll)

In [43]:
predhets=forecast_list$pred_ets$predhets
predets=forecast_list$pred_ets$predets
predhthetaf=forecast_list$pred_thetaf$predhthetaf
predthetaf=forecast_list$pred_thetaf$predthetaf
y_real=forecast_list$y_real

In [189]:
length5=matrix(0,ll,m)
for(k in 1:ll)
{
y = data_tstrain[[k]]
y_l=length(y)
loc = 1:length(y)
loc_m = as.integer(loc*m/length(y))
filt_d = data.frame(y,loc_m)
filt_0=filter(filt_d ,loc_m==0)
m_l=count(filt_0)
m_l=as.integer(m_l)
length5[k,]=c(0,1*m_l,2*m_l,3*m_l,4*m_l)
    }

### 进行数据模拟

In [43]:
library(gratis)
library(feasts)

Loading required package: fabletools


Attaching package: ‘fabletools’


The following object is masked from ‘package:forecast’:

    accuracy



Attaching package: ‘feasts’


The following objects are masked from ‘package:tsfeatures’:

    unitroot_kpss, unitroot_pp




In [44]:
my_features <- function(x) {
  output <- c(tsfeatures(x))
  output["long"] <- length(x)
  unlist(output)
}

In [57]:
n_g=50

In [ ]:
library(parallel)
gratislist=list()
n_g=50
for(i in 1:ll) 
{  
  datamatrix=matrix(0,length(data_tstrain[[i]]),n)  
  x <- generate_ts_with_target(n=n_g,
                               ts.length = length(Goldtstrain[[i]]),
                               freq = frequency(Goldtstrain[[i]]),
                               seasonal = 1,
                               features ="my_features",
                               selected.features = colnames(Goldf),
                               target = as.vector(Goldf[i,]))
  
  gratislist[[i]]=x
}       

In [ ]:
# clnum <- detectCores()
# cl <- makeCluster(getOption("cl.cores", 5))

# fun <- function(i) {
#   datamatrix=matrix(0,length(Goldtstrain[[i]]),n)  
#   x <- generate_ts_with_target(n=n,
#                                ts.length = length(Goldtstrain[[i]]),
#                                freq = frequency(Goldtstrain[[i]]),
#                                seasonal = 1,
#                                features ="my_features",
#                                selected.features = colnames(Goldf),
#                                target = as.vector(Goldf[i,]))
#   return(x)
# }

# # 导出'Goldtstrain', 'Goldf', 'my_features'到每个任务的环境中
# clusterExport(cl, c("Goldtstrain", "Goldf", "my_features","n",'generate_ts_with_target','tsfeatures'))

# gratislist <- parLapply(cl, 1:ll, fun)

# stopCluster(cl)

In [46]:
save(gratislist,file='gratislist_信心指数_50_12.RData')

In [51]:
loaddata=load('gratislist_信心指数_50_12.RData')
gratislist=eval(parse(text = loaddata))

In [52]:
gratislist[[1]]

,Series 1,Series 2,Series 3,Series 4,Series 5,Series 6,Series 7,Series 8,Series 9,Series 10,Series 11,Series 12,Series 13,Series 14,Series 15,Series 16,Series 17,Series 18,Series 19,Series 20,Series 21,Series 22,Series 23,Series 24,Series 25,Series 26,Series 27,Series 28,Series 29,Series 30,Series 31,Series 32,Series 33,Series 34,Series 35,Series 36,Series 37,Series 38,Series 39,Series 40,Series 41,Series 42,Series 43,Series 44,Series 45,Series 46,Series 47,Series 48,Series 49,Series 50
Jan 1,12.45135662,4.84501256,-7.0101697,-1.1410879,-12.4023771,3.787890e+00,-0.60444978,18.2879830,6.1072978,-3.97690358,4.252667410,8.90041162,2.795280902,0.22024018,-1.14708268,-8.13531620,2.19621268,-1.068878868,2.18051286,-9.816495e-01,-1.36816710,-5.14526224,1.786820485,-19.417029,-33.2141572,-2.28549398,4.2526129,-245.4912681,6.184939834,7.139633e+00,5.44383292,-1.43117450,-0.88745907,4.45249751,6.2367484,4.606766e+00,7.6737652,-0.96782958,-5.50156657,-1.769578999,8.75057213,0.29259466,19.11556051,6.7817851,-24.071249344,4.30452334,2.9860347,-10.138936725,-1.9501874,-0.6424806
Feb 1,8.40870870,-1.99976972,-3.0043259,12.6088927,-6.7805324,2.320016e+00,4.56170012,29.8241053,3.9503810,-4.98847791,4.955386005,3.46927175,-2.189879782,-3.22589414,-1.37832102,-11.70756832,7.75734569,-1.655701328,6.21898161,-1.570555e+01,3.70670981,0.83162555,-2.181604396,-24.606892,-41.4939816,-0.44763532,4.0395622,-225.0577361,4.257223306,4.279412e+00,1.96019190,1.08659206,-0.40190708,2.78896768,6.6067856,3.518018e+00,5.7004277,5.61378092,-3.22007045,-2.519324353,-2.01037981,-1.46346063,33.21259923,2.6735790,-16.173696272,4.26027031,2.6749747,-12.221367149,-7.5737800,-1.9233191
Mar 1,17.11123632,2.61439189,-16.1654677,12.4955206,-6.7470460,3.435139e+00,0.22503933,23.6240350,4.8570779,-7.62580385,5.097412447,5.31199761,-1.215038383,3.07004198,-0.71916772,-11.60057680,4.97091229,-0.568326172,2.74821440,-9.239728e+00,-0.81161179,-11.88489767,0.127070141,-42.094360,-35.6930024,-2.65355606,4.4208543,-196.0966957,6.054280605,3.933968e+00,1.35815756,3.61902563,1.05225996,5.03004429,4.9782604,7.438936e+00,2.5970507,-2.81675423,-4.86271887,-1.631423356,5.37052207,-0.76680160,18.51358879,6.1438150,-24.579570778,3.46817900,1.1667303,-9.249157857,-13.5436942,-3.6046742
Apr 1,8.76713593,2.96816125,-5.6950494,12.5663729,-21.8738435,1.527446e-01,-1.79400308,18.7442598,3.3566119,-8.79829802,4.833853904,-4.53127088,-3.093631902,0.16361199,-1.73106627,-6.50808607,3.32846889,1.369941743,6.03340073,-2.291575e+01,1.36290126,-5.78152598,-0.291038728,-25.317379,-36.4624294,-1.72917629,3.3761738,-196.3892934,7.685551535,-4.287359e-01,-1.01018301,-0.29981301,0.81299263,5.33438436,8.8491376,6.529055e+00,8.4962683,-3.64628825,-4.06938333,-3.927445160,5.16040401,-4.53894583,30.11388180,3.9067960,-31.972846903,2.75854096,-0.6151279,-6.502669555,-12.9654087,-3.3762735
May 1,15.58341045,-0.58094497,-13.3311247,-1.2763192,-12.6637135,-2.712369e-01,-4.01617291,19.6236773,-7.4306324,-14.34363329,3.161428659,-5.24580388,1.074483337,-0.95321322,-4.14068230,-5.53905598,-0.15619732,-0.077421143,6.30335383,-1.231621e+01,-3.52280721,-5.95353362,-1.150543756,-20.350115,-34.4077235,-1.99095952,5.9844830,-170.3696710,8.739791459,-1.446070e+00,1.19426359,-1.28482691,0.74898377,0.80046519,1.4947516,-1.473113e+00,-2.0584015,-2.73464158,-3.86120837,-2.858504783,-1.03717782,-9.12966744,20.22798623,6.0635653,-30.290697692,5.60439377,-0.7988690,-14.769803725,-14.0506969,-1.7549297
Jun 1,8.86257190,2.73945435,-10.7228254,5.2310383,-20.4125565,2.997037e-01,-2.88119399,17.7118599,-2.0321220,-18.68173506,0.939456458,-0.10140551,-0.333010224,-1.45350761,-3.86520218,-5.28046646,3.14291867,-0.458553949,4.31614470,-1.869092e+01,-1.15165311,-8.28290472,-2.651885777,-33.994951,-36.0385875,-0.06177287,1.4058669,-158.6115407,9.805938406,3.746435e-01,-5.23634835,1.44511750,0.71119616,1.71702527,0.4095499,-5.580354e-01,-1.2296818,-7.68860799,-5.05494573,1.255161798,3.06898781,-6.31835963,34.45554210,7.2315601,-34.347672669,

### 模拟数据预训练

In [ ]:
predetsy = array(0,dim = c(ll*n_g,5,4,6))
predhetsy = array(0,dim = c(ll*n_g,5,4,hh))
for(k in 1:ll){
    yall=gratislist[[k]]
    yall_n=dim(yall)[2]
    for(t in 1:yall_n)
    {
    y=yall[,t]
    y_pred = ts(y[(length(y)-hh+1):length(y)],end = end(y), frequency=frequency(y))
    y_past = ts(y[1:(length(y)-hh)],start = start(y), frequency=frequency(y))
    loc = 1:length(y_past)
    loc0 = as.integer(loc*5/length(y_past))
    d = data.frame(y_past,loc0)
    y1=filter(d,loc0==0)
    h=count(y1)
    m_l=h/4
    h=as.integer(h)
    m_l=as.numeric(m_l)
    for(i in 0:(m-1)){
        for(j in 0:(n-1)){
            Y = ts(y_past[f1((i*h+j*m_l)+1):length(y_past)],end = end(y_past), frequency=frequency(yall))
            M = ets(Y) 
            pd=forecast(M, h=hh)
            predhetsy[(k-1)*yall_n+t,(i+1),(j+1),] =pd$mean 
            predetsy[(k-1)*yall_n+t,(i+1),(j+1),] = forecast::accuracy(pd, y_pred)[2,1:6]
        }
    }
}
}

In [118]:
predthetafy = array(0,dim = c(ll*n_g,5,4,6))
predhthetafy = array(0,dim = c(ll*n_g,5,4,hh))
for(k in 1:ll){
    yall=gratislist[[k]]
    yall_n=dim(yall)[2]
    for(t in 1:yall_n)
    {
    y=yall[,t]
    y_pred = ts(y[(length(y)-hh+1):length(y)],end = end(y), frequency=frequency(y))
    y_past = ts(y[1:(length(y)-hh)],start = start(y), frequency=frequency(y))
    loc = 1:length(y_past)
    loc0 = as.integer(loc*5/length(y_past))
    d = data.frame(y_past,loc0)
    y1=filter(d,loc0==0)
    h=count(y1)
    m_l=h/4
    h=as.integer(h)
    m_l=as.numeric(m_l)
    for(i in 0:(m-1)){
        for(j in 0:(n-1)){
            Y = ts(y_past[f1((i*h+j*m_l)+1):length(y_past)],end = end(y_past), frequency=frequency(yall))
            M = thetaf(Y,h=hh) 
            pd=forecast(M, h=hh)
            predthetafy[(k-1)*yall_n+t,(i+1),(j+1),] = forecast::accuracy(pd, y_pred)[2,1:6]
            predhthetafy[(k-1)*yall_n+t,(i+1),(j+1),] = pd$mean
        }
    }
}
}

Warning message in thetaf(Y, h = hh):
“Seasonal indexes close to zero. Using non-seasonal Theta method”
Warning message in thetaf(Y, h = hh):
“Seasonal indexes close to zero. Using non-seasonal Theta method”
Warning message in thetaf(Y, h = hh):
“Seasonal indexes close to zero. Using non-seasonal Theta method”
Warning message in thetaf(Y, h = hh):
“Seasonal indexes close to zero. Using non-seasonal Theta method”
Warning message in thetaf(Y, h = hh):
“Seasonal indexes close to zero. Using non-seasonal Theta method”
Warning message in thetaf(Y, h = hh):
“Seasonal indexes close to zero. Using non-seasonal Theta method”
Warning message in thetaf(Y, h = hh):
“Seasonal indexes close to zero. Using non-seasonal Theta method”
Warning message in thetaf(Y, h = hh):
“Seasonal indexes close to zero. Using non-seasonal Theta method”
Warning message in thetaf(Y, h = hh):
“Seasonal indexes close to zero. Using non-seasonal Theta method”
Warning message in thetaf(Y, h = hh):
“Seasonal indexes close to

### 训练数据特征提取

In [73]:
gratistrainlist=list()
long=matrix(0,ll*n,1)
for(i in 1:ll)
    {
    n1=dim(gratislist[[i]])[1]
    n2=dim(gratislist[[i]])[2]
    gratisdata=gratislist[[i]]
    x=gratisdata[,1]
    da1=ts(x[1:(length(x)-hh)],start=start(x),frequency=frequency(x))
    long[(i-1)*n+1]=length(da1)
    for(j in 2:n2)
        {
        x=gratisdata[,j]
        da=ts(x[1:(length(x)-hh)],start=start(x),frequency=frequency(x))
        da1=cbind(da1,da)
        long[(i-1)*n+j]=length(da)
    }
    new_colnames <- paste("Series", 1:ncol(da1))
    # 设置列名
    colnames(da1) <- new_colnames
    gratistrainlist[[i]]=da1
}

In [75]:
gratistrainlist[[1]]

,Series 1,Series 2,Series 3,Series 4,Series 5,Series 6,Series 7,Series 8,Series 9,Series 10,Series 11,Series 12,Series 13,Series 14,Series 15,Series 16,Series 17,Series 18,Series 19,Series 20,Series 21,Series 22,Series 23,Series 24,Series 25,Series 26,Series 27,Series 28,Series 29,Series 30,Series 31,Series 32,Series 33,Series 34,Series 35,Series 36,Series 37,Series 38,Series 39,Series 40,Series 41,Series 42,Series 43,Series 44,Series 45,Series 46,Series 47,Series 48,Series 49,Series 50
Jan 1,12.45135662,4.84501256,-7.0101697,-1.1410879,-12.4023771,3.787890e+00,-0.60444978,18.2879830,6.1072978,-3.97690358,4.252667410,8.90041162,2.795280902,0.22024018,-1.14708268,-8.13531620,2.19621268,-1.068878868,2.18051286,-9.816495e-01,-1.36816710,-5.14526224,1.786820485,-19.417029,-33.214157,-2.28549398,4.2526129,-245.4912681,6.18493983,7.139633e+00,5.4438329,-1.43117450,-0.88745907,4.45249751,6.2367484,4.606766e+00,7.6737652,-0.96782958,-5.50156657,-1.769578999,8.75057213,0.29259466,19.11556051,6.7817851,-24.071249344,4.30452334,2.9860347,-10.138936725,-1.9501874,-0.6424806
Feb 1,8.40870870,-1.99976972,-3.0043259,12.6088927,-6.7805324,2.320016e+00,4.56170012,29.8241053,3.9503810,-4.98847791,4.955386005,3.46927175,-2.189879782,-3.22589414,-1.37832102,-11.70756832,7.75734569,-1.655701328,6.21898161,-1.570555e+01,3.70670981,0.83162555,-2.181604396,-24.606892,-41.493982,-0.44763532,4.0395622,-225.0577361,4.25722331,4.279412e+00,1.9601919,1.08659206,-0.40190708,2.78896768,6.6067856,3.518018e+00,5.7004277,5.61378092,-3.22007045,-2.519324353,-2.01037981,-1.46346063,33.21259923,2.6735790,-16.173696272,4.26027031,2.6749747,-12.221367149,-7.5737800,-1.9233191
Mar 1,17.11123632,2.61439189,-16.1654677,12.4955206,-6.7470460,3.435139e+00,0.22503933,23.6240350,4.8570779,-7.62580385,5.097412447,5.31199761,-1.215038383,3.07004198,-0.71916772,-11.60057680,4.97091229,-0.568326172,2.74821440,-9.239728e+00,-0.81161179,-11.88489767,0.127070141,-42.094360,-35.693002,-2.65355606,4.4208543,-196.0966957,6.05428060,3.933968e+00,1.3581576,3.61902563,1.05225996,5.03004429,4.9782604,7.438936e+00,2.5970507,-2.81675423,-4.86271887,-1.631423356,5.37052207,-0.76680160,18.51358879,6.1438150,-24.579570778,3.46817900,1.1667303,-9.249157857,-13.5436942,-3.6046742
Apr 1,8.76713593,2.96816125,-5.6950494,12.5663729,-21.8738435,1.527446e-01,-1.79400308,18.7442598,3.3566119,-8.79829802,4.833853904,-4.53127088,-3.093631902,0.16361199,-1.73106627,-6.50808607,3.32846889,1.369941743,6.03340073,-2.291575e+01,1.36290126,-5.78152598,-0.291038728,-25.317379,-36.462429,-1.72917629,3.3761738,-196.3892934,7.68555154,-4.287359e-01,-1.0101830,-0.29981301,0.81299263,5.33438436,8.8491376,6.529055e+00,8.4962683,-3.64628825,-4.06938333,-3.927445160,5.16040401,-4.53894583,30.11388180,3.9067960,-31.972846903,2.75854096,-0.6151279,-6.502669555,-12.9654087,-3.3762735
May 1,15.58341045,-0.58094497,-13.3311247,-1.2763192,-12.6637135,-2.712369e-01,-4.01617291,19.6236773,-7.4306324,-14.34363329,3.161428659,-5.24580388,1.074483337,-0.95321322,-4.14068230,-5.53905598,-0.15619732,-0.077421143,6.30335383,-1.231621e+01,-3.52280721,-5.95353362,-1.150543756,-20.350115,-34.407724,-1.99095952,5.9844830,-170.3696710,8.73979146,-1.446070e+00,1.1942636,-1.28482691,0.74898377,0.80046519,1.4947516,-1.473113e+00,-2.0584015,-2.73464158,-3.86120837,-2.858504783,-1.03717782,-9.12966744,20.22798623,6.0635653,-30.290697692,5.60439377,-0.7988690,-14.769803725,-14.0506969,-1.7549297
Jun 1,8.86257190,2.73945435,-10.7228254,5.2310383,-20.4125565,2.997037e-01,-2.88119399,17.7118599,-2.0321220,-18.68173506,0.939456458,-0.10140551,-0.333010224,-1.45350761,-3.86520218,-5.28046646,3.14291867,-0.458553949,4.31614470,-1.869092e+01,-1.15165311,-8.28290472,-2.651885777,-33.994951,-36.038587,-0.06177287,1.4058669,-158.6115407,9.80593841,3.746435e-01,-5.2363484,1.44511750,0.71119616,1.71702527,0.4095499,-5.580354e-01,-1.2296818,-7.68860799,-5.05494573,1.255161798,3.06898781,-6.31835963,34.45554210,7.2315601,-34.347672669,3.99739670,-0.4278

In [76]:
datatsy=list()
datatstrainy=list()
i=1
da=gratistrainlist[[i]]
f=tsfeatures(da)
datay=f

In [89]:
datatsy=list()
datatstrainy=list()
i=1
da=gratistrainlist[[i]]
f=tsfeatures(da)
f$long=dim(da)[1]
datay=f
for( i in 2:ll)
    {
    da=gratistrainlist[[i]]
    f=tsfeatures(da)
    f$long=dim(da)[1]
    datay=rbind(datay,f)
}


In [90]:
library("xgboost")
library("Matrix")
library('Ckmeans.1d.dp')
library(lightgbm)


Attaching package: ‘xgboost’


The following object is masked from ‘package:dplyr’:

    slice



Attaching package: ‘lightgbm’


The following object is masked from ‘package:xgboost’:

    slice


The following object is masked from ‘package:dplyr’:

    slice




In [91]:
# etsxgbreg <- readRDS("etsxgbreg.rds")
# etsxgbcls <- readRDS("etsxgbcls.rds")
# etslgbreg <- readRDS.lgb.Booster("etslgbreg.rds")
# etslgbcls <- readRDS.lgb.Booster("etslgbcls.rds")

In [92]:
# thetafxgbreg <- readRDS("thetafxgbreg.rds")
# thetafxgbcls <- readRDS("thetafxgbcls.rds")
# thetaflgbreg <- readRDS.lgb.Booster("thetaflgbreg.rds")
# thetaflgbcls <- readRDS.lgb.Booster("thetaflgbcls.rds")

### 用模拟数据进行训练

In [119]:
meanunique=function(x)
    {
    mean(unique(x))
}

In [120]:
whichmin<-function(x){
    minx=min(x[x>0])
    loc=which(x==minx)[1]-1
    loc
}

In [130]:
realbest_construct<-function(MASE)
    {
realbestmean=matrix(0,dim(MASE)[1],1)
for(i in seq(1,dim(MASE)[1]))
    {
    count=MASE[i,,]
    line=apply(count,1,meanunique)
    if (max(line,na.rm = TRUE)==0){
        realbestmean[i,]=0
        }
    else
        {
        line[is.na(line)]=100 
        realbestmean[i,]= whichmin(line)
    }
    }
    return(realbestmean)
    }
    

In [131]:
MASEets

, , 1

                [,1]         [,2]         [,3]         [,4]         [,5]
   [1,]    2.2716979    1.9397970 1.326059e+00    1.9125910 1.748705e+00
   [2,]    5.9767524    6.4688864 5.644375e+00    3.4284612 3.402609e+00
   [3,]    4.7371570    3.7416531 4.035171e+00    3.1569976 3.752383e+00
   [4,]    1.0635774    1.6692353 2.240784e+00    2.0364381 7.524360e-01
   [5,]    4.7514836    3.1745368 2.454463e+00    2.0768583 9.709968e-01
   [6,]   39.4825889   47.5663498 3.297214e+01    5.6641780 1.265905e+00
   [7,]   94.0466780   11.0867057 4.450363e+01    4.6805458 3.278273e+00
   [8,]   10.4813287    9.1438452 6.721003e+00    6.9064509 1.468270e+01
   [9,]   20.2199973   19.6632975 2.205490e+01   77.1461852 1.530793e+00
  [10,]    7.0291232    4.8772007 8.211254e+00    7.1091753 7.169693e-01
  [11,]    1.1508336    0.7567829 6.662212e-01    0.6919498 1.252285e+00
  [12,]    5.5772292    6.4103034 5.966297e+00   50.3550013 1.467265e+00
  [13,]   49.5689926    0.7030350 1.363863e+00    1.1074408 4.779595e-01
  [14,]    6.2035648    1.5560592 1.862789e+00    2.2870158 7.496244e-01
  [15,]    4.5504389    1.2049910 3.107657e+00    3.7245448 1.836495e+00
  [16,]    7.1332374    5.8024865 9.540241e+00   71.1648690 4.217999e+00
  [17,]    1.8234686    1.6440456 1.593532e+00    1.6155852 2.911524e+01
  [18,]    3.6857355    5.2639678 1.440917e+00    1.4017694 9.134548e-01
  [19,]    0.7196139    0.8449145 8.765611e-01    0.4381781 6.881543e-01
  [20,]    8.2962923    6.9187377 5.309434e+00    3.9471422 6.524768e-01
  [21,]   46.0159277   20.7831481 3.184189e+01    5.6528590 9.314031e-01
  [22,]    2.6623563    2.2686736 8.404884e+00    7.4073476 2.640730e+00
  [23,]    9.5511266   11.1354281 8.686977e+00    1.7111304 5.546487e+00
  [24,]    6.7997058    5.5632746 5.614616e+00    3.8461121 2.250847e+00
  [25,]    0.5214413    0.6239216 5.227834e-01    0.4432154 5.142804e-01
  [26,]    1.2743179    1.6507078 1.305987e+00    1.2298029 7.181414e-01
  [27,]    3.5233640    3.2417254 1.988215e+00    1.4516889 9.144352e-01
  [28,]    0.5995437    1.2452368 3.847155e+01    0.8861776 1.737507e-01
  [29,]    4.1649056    2.2136039 2.381771e+00    2.1891823 9.032856e-01
  [30,]   26.2637291   44.5470844 1.041383e+02    2.8717340 1.335213e+00
  [31,]   11.7450279    8.8756015 8.394434e+00    6.6094926 3.017963e+00
  [32,]    7.1815426    0.8843353 9.337010e-01    1.2267059 7.227938e-01
  [33,]    3.1561579    1.6957696 1.308773e+00    0.8129035 1.124462e+00
  [34,]    3.6145815    5.8399587 8.096622e+00    3.0372116 7.589123e-01
  [35,]    1.5200694    1.4812127 1.477495e+00    1.2905638 7.547732e-01
  [36,]   28.5661246   27.6506002 3.704918e+01   14.5833189 1.706237e+00
  [37,]   72.3996223   22.1613606 8.742249e+00    3.1705163 1.109438e+00
  [38,]    3.0433649    2.9928880 1.413159e+01    1.8927934 1.990499e+00
  [39,]    1.5698781    1.6852057 2.183820e+00    4.0070955 1.803288e+00
  [40,]    1.8263220    1.5850110 1.646370e+00    1.8893776 5.589005e+00
  [41,]    8.7693026    7.0455857 5.749803e+00   27.7679091 1.957326e+01
  [42,]    6.8378118   13.1431949 2.622290e+00    1.7144789 1.235337e+00
  [43,]   11.2203265   13.2116156 6.081636e+00    3.2467846 2.384285e+00
  [44,]    9.3865960   11.2721826 7.076207e+00    5.6280919 6.090943e+00
  [45,]   40.5819758    8.4596146 1.026190e+01    4.5164069 3.626621e+00
  [46,]    1.6137056    2.0834791 2.508715e+00    2.7854885 1.823252e+00
  [47,]   17.6918935   22.0496316 2.947404e+01   23.4885449 2.978042e+00
  [48,]    2.2345574    4.2487642 1.033167e+00    0.7437935 9.118651e-01
  [49,]    3.1631454    2.8371427 4.285678e+00   18.3485174 1.354620e+00
  [50,]   26.6077628   35.8004322 8.327409e+00   13.0553717 2.071720e+00
  [51,]   13.5991897    3.0607302 5.197340e+00    7.5114184 4.895734e+01
  [52,]    6.2676002    6.1943941 8.711612e+00   17.9551331 5.015422e-01
  [53,]    1.3625859    1.1382321 9.060134e-01    2.2629157 1.799529e+00
  [54,]    1.2021829    1.1573594 1.012817e+00    1

In [132]:
MASEets=predetsy[,,,6]
MASEthetaf=predthetafy[,,,6]
realbestmeanthetaf=realbest_construct(MASEthetaf)
realbestmeanets=realbest_construct(MASEets)

In [134]:
# 假设你的数组名为MASEets
# 首先检查整个数组中是否存在缺失值
missing_values <- is.na(MASEets)

# 然后对第一个维度（2200）应用sum函数，统计每个元素中的缺失值数量
missing_in_dim1 <- apply(missing_values, 1, sum)

# 找出含有缺失值的元素的索引
indices_with_NA <- which(missing_in_dim1 > 0)

# 打印含有缺失值的元素的索引
print(indices_with_NA)


 [1] 2051 2052 2053 2054 2055 2056 2057 2058 2059 2060 2061 2062 2063 2064 2065
[16] 2066 2067 2068 2069 2070 2071 2072 2073 2074 2075 2076 2077 2078 2079 2080
[31] 2081 2082 2083 2084 2085 2086 2087 2088 2089 2090 2091 2092 2093 2094 2095
[46] 2096 2097 2098 2099 2100


In [136]:
train_data=datay
train_label=realbestmeanets
train_label1=realbestmeanthetaf

In [138]:
dtrain20 <- xgb.DMatrix(data = as.matrix(train_data),label = as.matrix(train_label)) 
dtrain21 <- xgb.DMatrix(data = as.matrix(train_data),label = as.matrix(as.factor(train_label)))

In [139]:
etsxgbreg  <- xgboost(data = dtrain20, nround=100)

[1]	train-rmse:2.068637 
[2]	train-rmse:1.706076 
[3]	train-rmse:1.483989 
[4]	train-rmse:1.332198 
[5]	train-rmse:1.250241 
[6]	train-rmse:1.185504 
[7]	train-rmse:1.123717 
[8]	train-rmse:1.095725 
[9]	train-rmse:1.051101 
[10]	train-rmse:1.011013 
[11]	train-rmse:0.991487 
[12]	train-rmse:0.978730 
[13]	train-rmse:0.969538 
[14]	train-rmse:0.943121 
[15]	train-rmse:0.916803 
[16]	train-rmse:0.910257 
[17]	train-rmse:0.894049 
[18]	train-rmse:0.887237 
[19]	train-rmse:0.863205 
[20]	train-rmse:0.849727 
[21]	train-rmse:0.836419 
[22]	train-rmse:0.817234 
[23]	train-rmse:0.802354 
[24]	train-rmse:0.787810 
[25]	train-rmse:0.781930 
[26]	train-rmse:0.771993 
[27]	train-rmse:0.767459 
[28]	train-rmse:0.754061 
[29]	train-rmse:0.729180 
[30]	train-rmse:0.714756 
[31]	train-rmse:0.698700 
[32]	train-rmse:0.690849 
[33]	train-rmse:0.680990 
[34]	train-rmse:0.675933 
[35]	train-rmse:0.654559 
[36]	train-rmse:0.645958 
[37]	train-rmse:0.627356 
[38]	train-rmse:0.623240 
[39]	train-rmse:0.603

In [140]:
etsxgbcls  <- xgboost(data = dtrain21, nround=100, objective='multi:softmax',num_class=5)

[1]	train-mlogloss:1.436161 
[2]	train-mlogloss:1.321067 
[3]	train-mlogloss:1.225375 
[4]	train-mlogloss:1.159956 
[5]	train-mlogloss:1.100425 
[6]	train-mlogloss:1.048339 
[7]	train-mlogloss:1.004112 
[8]	train-mlogloss:0.969431 
[9]	train-mlogloss:0.931816 
[10]	train-mlogloss:0.889555 
[11]	train-mlogloss:0.863221 
[12]	train-mlogloss:0.838038 
[13]	train-mlogloss:0.810054 
[14]	train-mlogloss:0.793884 
[15]	train-mlogloss:0.768583 
[16]	train-mlogloss:0.740825 
[17]	train-mlogloss:0.705700 
[18]	train-mlogloss:0.690993 
[19]	train-mlogloss:0.677707 
[20]	train-mlogloss:0.658033 
[21]	train-mlogloss:0.644720 
[22]	train-mlogloss:0.627657 
[23]	train-mlogloss:0.611879 
[24]	train-mlogloss:0.600857 
[25]	train-mlogloss:0.590404 
[26]	train-mlogloss:0.573025 
[27]	train-mlogloss:0.553856 
[28]	train-mlogloss:0.546442 
[29]	train-mlogloss:0.527733 
[30]	train-mlogloss:0.510675 
[31]	train-mlogloss:0.496323 
[32]	train-mlogloss:0.479933 
[33]	train-mlogloss:0.468171 
[34]	train-mlogloss

In [141]:
dtrain22 <- lgb.Dataset(data = as.matrix(train_data),label = as.matrix(train_label))
dtrain23 <- lgb.Dataset(data = as.matrix(train_data),label = as.matrix(as.factor(train_label)))

In [148]:
# 定义参数列表
params <- list(
  objective = "regression",
  metric = "rmse"
)

etslgbreg  <- lgb.train(data = dtrain22,nrounds = 100,params = params)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008384 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3865
[LightGBM] [Info] Number of data points in the train set: 2200, number of used features: 18
[LightGBM] [Info] Start training from score 2.756818


In [151]:
# 定义参数列表
params <- list(
  objective = 'multiclass',
  num_class = 5,
  num_iterations = 100  # 使用 num_iterations 代替 nrounds
)

# 使用参数列表进行训练
etslgbcls <- lgb.train(
  data = dtrain23,
  params = params
)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003364 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3865
[LightGBM] [Info] Number of data points in the train set: 2200, number of used features: 18
[LightGBM] [Info] Start training from score -2.097791
[LightGBM] [Info] Start training from score -2.392908
[LightGBM] [Info] Start training from score -2.194954
[LightGBM] [Info] Start training from score -1.364711
[LightGBM] [Info] Start training from score -0.869667


### thetaf训练

In [143]:
dtrain20 <- xgb.DMatrix(data = as.matrix(train_data),label = as.matrix(train_label1)) 
dtrain21 <- xgb.DMatrix(data = as.matrix(train_data),label = as.matrix(as.factor(train_label1)))

In [144]:
thetafxgbreg  <- xgboost(data = dtrain20, nround=100)

[1]	train-rmse:2.068637 
[2]	train-rmse:1.706076 
[3]	train-rmse:1.483989 
[4]	train-rmse:1.332198 
[5]	train-rmse:1.250241 
[6]	train-rmse:1.185504 
[7]	train-rmse:1.123717 
[8]	train-rmse:1.095725 
[9]	train-rmse:1.051101 
[10]	train-rmse:1.011013 
[11]	train-rmse:0.991487 
[12]	train-rmse:0.978730 
[13]	train-rmse:0.969538 
[14]	train-rmse:0.943121 
[15]	train-rmse:0.916803 
[16]	train-rmse:0.910257 
[17]	train-rmse:0.894049 
[18]	train-rmse:0.887237 
[19]	train-rmse:0.863205 
[20]	train-rmse:0.849727 
[21]	train-rmse:0.836419 
[22]	train-rmse:0.817234 
[23]	train-rmse:0.802354 
[24]	train-rmse:0.787810 
[25]	train-rmse:0.781930 
[26]	train-rmse:0.771993 
[27]	train-rmse:0.767459 
[28]	train-rmse:0.754061 
[29]	train-rmse:0.729180 
[30]	train-rmse:0.714756 
[31]	train-rmse:0.698700 
[32]	train-rmse:0.690849 
[33]	train-rmse:0.680990 
[34]	train-rmse:0.675933 
[35]	train-rmse:0.654559 
[36]	train-rmse:0.645958 
[37]	train-rmse:0.627356 
[38]	train-rmse:0.623240 
[39]	train-rmse:0.603

In [145]:
thetafxgbcls  <- xgboost(data = dtrain21, nround=100, objective='multi:softmax',num_class=5)

[1]	train-mlogloss:1.436161 
[2]	train-mlogloss:1.321067 
[3]	train-mlogloss:1.225375 
[4]	train-mlogloss:1.159956 
[5]	train-mlogloss:1.100425 
[6]	train-mlogloss:1.048339 
[7]	train-mlogloss:1.004112 
[8]	train-mlogloss:0.969431 
[9]	train-mlogloss:0.931816 
[10]	train-mlogloss:0.889555 
[11]	train-mlogloss:0.863221 
[12]	train-mlogloss:0.838038 
[13]	train-mlogloss:0.810054 
[14]	train-mlogloss:0.793884 
[15]	train-mlogloss:0.768583 
[16]	train-mlogloss:0.740825 
[17]	train-mlogloss:0.705700 
[18]	train-mlogloss:0.690993 
[19]	train-mlogloss:0.677707 
[20]	train-mlogloss:0.658033 
[21]	train-mlogloss:0.644720 
[22]	train-mlogloss:0.627657 
[23]	train-mlogloss:0.611879 
[24]	train-mlogloss:0.600857 
[25]	train-mlogloss:0.590404 
[26]	train-mlogloss:0.573025 
[27]	train-mlogloss:0.553856 
[28]	train-mlogloss:0.546442 
[29]	train-mlogloss:0.527733 
[30]	train-mlogloss:0.510675 
[31]	train-mlogloss:0.496323 
[32]	train-mlogloss:0.479933 
[33]	train-mlogloss:0.468171 
[34]	train-mlogloss

In [146]:
dtrain22 <- lgb.Dataset(data = as.matrix(train_data),label = as.matrix(train_label1))
dtrain23 <- lgb.Dataset(data = as.matrix(train_data),label = as.matrix(as.factor(train_label1)))

In [152]:
# 定义参数列表
params <- list(
  objective = "regression",
  metric = "rmse"
)

thetaflgbreg  <- lgb.train(data = dtrain22,nrounds = 100,params = params)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007190 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3865
[LightGBM] [Info] Number of data points in the train set: 2200, number of used features: 18
[LightGBM] [Info] Start training from score 2.756818


In [154]:
# 定义参数列表
params <- list(
  objective = 'multiclass',
  num_class = 5,
  num_iterations = 100  # 使用 num_iterations 代替 nrounds
)

# 使用参数列表进行训练
thetaflgbcls <- lgb.train(
  data = dtrain23,
  params = params
)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003603 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3865
[LightGBM] [Info] Number of data points in the train set: 2200, number of used features: 18
[LightGBM] [Info] Start training from score -2.097791
[LightGBM] [Info] Start training from score -2.392908
[LightGBM] [Info] Start training from score -2.194954
[LightGBM] [Info] Start training from score -1.364711
[LightGBM] [Info] Start training from score -0.869667


## 预测与输出误差

In [158]:
alldatalgb <- lgb.Dataset(data = as.matrix(data_features))
alldataxgb <- xgb.DMatrix(data = as.matrix(data_features))

In [160]:
xgbregets=predict(etsxgbreg,alldataxgb)
xgbclsets=predict(etsxgbcls,alldataxgb)
lgbregets=predict(etslgbreg,as.matrix(data_features))
lgbclsets=predict(etslgbcls,as.matrix(data_features))

In [162]:
xgbregthetaf=predict(thetafxgbreg,alldataxgb)
xgbclsthetaf=predict(thetafxgbcls,alldataxgb)
lgbregthetaf=predict(thetaflgbreg,as.matrix(data_features))
lgbclsthetaf=predict(thetaflgbcls,as.matrix(data_features))

In [166]:
lgbclsm=matrix(lgbclsets,5,ll)
lgbclsetsr=matrix(0,ll,1)
for(i in 1:ll)
    {
    lgbclsetsr[i,]=which.max(lgbclsm[,i])
}

In [169]:
lgbclsm=matrix(lgbclsthetaf,5,ll)
lgbclsthetafr=matrix(0,ll,1)
for(i in 1:ll)
    {
    lgbclsthetafr[i,]=which.max(lgbclsm[,i])
}

In [171]:
round10=function(x)
    {
    l=round(x)
    for(i in 1:length(l))
    {
    if(l[i]>5)
        {
        l[i]=5
    }
    if(l[i]<1)
    {
        l[i]=1
    }
}
    l
}

In [182]:
ets_pr=matrix(0,ll,4)
ets_pr[,1]=round10(xgbregets+1)
ets_pr[,2]=xgbclsets+1
ets_pr[,3]=round10(lgbregets+1)
ets_pr[,4]=lgbclsetsr

In [183]:
thetaf_pr=matrix(0,ll,4)
thetaf_pr[,1]=round10(xgbregthetaf+1)
thetaf_pr[,2]=xgbclsthetaf+1
thetaf_pr[,3]=round10(lgbregthetaf+1)
thetaf_pr[,4]=lgbclsthetafr

In [184]:
meanpre=function(pred_array,opt_pr,h,testindex)
    {
    data_length=length(testindex)
    meanpreh=matrix(0,data_length,h)
    for (i in 1:data_length){
    index=testindex[i]
    loc=opt_pr[index]
    predh=pred_array[index,loc,,]
    predh=unique(predh)
    meanpreh[i,]=apply(predh,2,mean)
}
    return(meanpreh)
}

In [194]:
predres=function(pred_matrix,data,real_data,opt_pr,lengthmatrix,h,testindex)
    {
    data_length=length(testindex)
    pred_res=matrix(0,data_length,6)
    model=ets(data[[1]])
    fore_l=forecast(model,h=h)
    freq=frequency(data[[1]])
    for (i in 1:data_length){
    index=testindex[i]
    loc=opt_pr[index]
    start=lengthmatrix[i,loc]
    y_all=data[[index]]
    y=y_all[start:length(y_all)]
    fore_l$x=ts(y,frequency=freq,end=end(y_all))
    real=real_data[[index]]
    fore_l$mean=ts(pred_matrix[i,],start=start(real),frequency=freq)
    res=accuracy(fore_l,real)
    pred_res[i,]=res[2,1:6]
    }
    return(pred_res)
    }

In [197]:
testindex=c(1:ll)
lengthtest=length5[testindex,]
etsxgbcls=meanpre(predhets,ets_pr[,1],hh,testindex)
etsxgbreg=meanpre(predhets,ets_pr[,2],hh,testindex)
etslgbcls=meanpre(predhets,ets_pr[,3],hh,testindex)
etslgbreg=meanpre(predhets,ets_pr[,4],hh,testindex)
etsxgbclsres=predres(etsxgbcls,data_tstrain,y_real,ets_pr[,1],lengthtest,hh,testindex)
etsxgbregres=predres(etsxgbreg,data_tstrain,y_real,ets_pr[,2],lengthtest,hh,testindex)
etslgbclsres=predres(etslgbcls,data_tstrain,y_real,ets_pr[,3],lengthtest,hh,testindex)
etslgbregres=predres(etslgbreg,data_tstrain,y_real,ets_pr[,4],lengthtest,hh,testindex)

In [200]:
thetafxgbcls=meanpre(predhthetaf,thetaf_pr[,1],hh,testindex)
thetafxgbreg=meanpre(predhthetaf,thetaf_pr[,2],hh,testindex)
thetaflgbcls=meanpre(predhthetaf,thetaf_pr[,3],hh,testindex)
thetaflgbreg=meanpre(predhthetaf,thetaf_pr[,4],hh,testindex)
thetafxgbclsres=predres(thetafxgbcls,data_tstrain,y_real,thetaf_pr[,1],lengthtest,hh,testindex)
thetafxgbregres=predres(thetafxgbreg,data_tstrain,y_real,thetaf_pr[,2],lengthtest,hh,testindex)
thetaflgbclsres=predres(thetaflgbcls,data_tstrain,y_real,thetaf_pr[,3],lengthtest,hh,testindex)
thetaflgbregres=predres(thetaflgbreg,data_tstrain,y_real,thetaf_pr[,4],lengthtest,hh,testindex)

In [199]:
apply(predets[,1,1,],2,mean)
apply(etsxgbclsres,2,mean)
apply(etsxgbregres,2,mean)
apply(etslgbclsres,2,mean)
apply(etslgbregres,2,mean)

[1] 1.724627 2.212382 1.957473 1.758364 1.998536 1.565424

[1] 1.756760 2.356424 2.062898 1.794217 2.109036 1.448435

[1] 1.850976 2.503484 2.202459 1.890433 2.250839 1.294785

[1] 1.811721 2.459538 2.165601 1.850666 2.212901 1.348099

[1] 1.786961 2.404576 2.120009 1.823440 2.165115 1.488895

In [201]:
apply(predthetaf[,1,1,],2,mean)
apply(thetafxgbclsres,2,mean)
apply(thetafxgbregres,2,mean)
apply(thetaflgbclsres,2,mean)
apply(thetaflgbregres,2,mean)

[1] 1.388740 1.743146 1.529509 1.410952 1.557603 1.186096

[1] 1.473611 1.816642 1.595257 1.500380 1.627212 1.100888

[1] 1.741423 2.104953 1.850294 1.774701 1.888284 1.052622

[1] 1.657858 2.023232 1.777988 1.689465 1.814590 1.077955

[1] 1.478229 1.836033 1.610562 1.502899 1.641065 1.080259

In [145]:
alltest=matrix(0,10,6)
alltest[1,]=apply(predets[,1,1,],2,mean)
alltest[2,]=apply(etsxgbclsres,2,mean)
alltest[3,]=apply(etsxgbregres,2,mean)
alltest[4,]=apply(etslgbclsres,2,mean)
alltest[5,]=apply(etslgbregres,2,mean)

alltest[6,]=apply(predthetaf[,1,1,],2,mean)
alltest[7,]=apply(thetafxgbclsres,2,mean)
alltest[8,]=apply(thetafxgbregres,2,mean)
alltest[9,]=apply(thetaflgbclsres,2,mean)
alltest[10,]=apply(thetaflgbregres,2,mean)

In [146]:
write.csv(alltest,'Imports_gratis_12.csv')